In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold
from sklearn.inspection import permutation_importance # 1002


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')
%cd 'drive/Shareddrives/CSCI 461/processed'
#%ls processed/*

In [ ]:
# pred = model.predict()
splits = kfold.split(X, y)
imp_by_cat = {key: [0.0,0.0] for key in CATEGORIES}    # keys: categories, values: [running total of permutation importance, count]
# imp_by_cat = {}
for n,(train_index, test_index) in enumerate(splits):   # each of k training/test sets
  r = permutation_importance(model, myX_ns.iloc[test_index, :], y.iloc[test_index],
                              n_repeats=30,     # can change this
                              random_state=0)
  # model.fit(X.iloc[train_index, :-1], y.iloc[train_index])
  # pred = model.predict(X.iloc[test_index, :-1])
  # for i in y[test_index]: y_truth.append(i)
  # for i in pred: y_pred.append(i)


  for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
      feat = lin_svc.feature[i]
      cat = feature_cat[feat]
      imp = r.importances_mean[i]
      # if cat in imp_by_cat:
      imp_by_cat[cat][0] += imp   # sum of means
      imp_by_cat[cat][1] += 1.0   # count
      # else:
      #   imp_by_cat[cat] = [0.0,0.0]
      print(f"{feat:<8}\t"
            f"{cat:<8}\t"
            f"{imp:.3f}\t"
            f" +/- {r.importances_std[i]:.3f}")

radar_input = {key: imp_by_cat[key][0]/imp_by_cat[key][1] if imp_by_cat[key][1] > 0 else 0 for key in imp_by_cat.keys()}    # keys: categories, values: mean importance in that category


In [ ]:
# example radar graph

import plotly.graph_objects as go

fig = go.Figure(layout_title_text="SVM Feature Importance")
fig.add_trace(go.Scatterpolar(
      r=list(radar_input.values()),
      theta=CATEGORIES,
      fill='toself',
      name='SVM'
))
# fig.add_trace(go.Scatterpolar(
#       r=[4, 3, 2.5, 1, 2],
#       theta=CATEGORIES,
#       fill='toself',
#       name='Product B'
# ))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, max_val]
    )),
  showlegend=False
)

fig.show()
